In [27]:
%load_ext autoreload
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload
import xes

In [3]:
activities_xes = xes.xes_to_dataframes(
    filepath="data\Activities of daily living of several individuals_1_all\data\edited_hh102_labour.xes\edited_hh102_labour.xes",
    keep_original=True
)

In [6]:
df = activities_xes[0]["events"]
df

,concept:name,lifecycle:transition,time:timestamp,Column_4
0,Start,start,2011-06-15 00:06:32+02:00,Start
1,Start,complete,2011-06-15 00:06:32+02:00,Start
2,sleep,start,2011-06-15 00:06:32+02:00,sleep
3,sleep,complete,2011-06-15 03:38:29+02:00,sleep
4,toilet,start,2011-06-15 03:38:44+02:00,toilet
...,...,...,...,...
77,personalhygiene,complete,2011-06-15 23:19:40+02:00,personalhygiene
78,sleep,start,2011-06-15 23:20:09+02:00,sleep
79,sleep,complete,2011-06-16 07:00:40+02:00,sleep
80,End,start,2011-06-16 07:00:40+02:00,End


In [15]:
def add_seconds_since_midnight(df, column_name="time:timestamp"):
    series = df[column_name]
    df["seconds_since_midnight"] = ((series - series.dt.normalize()) / pd.Timedelta("1 second")).astype(int)
    return df

In [16]:
add_seconds_since_midnight(df)

,concept:name,lifecycle:transition,time:timestamp,Column_4,seconds_since_midnight
0,Start,start,2011-06-15 00:06:32+02:00,Start,392
1,Start,complete,2011-06-15 00:06:32+02:00,Start,392
2,sleep,start,2011-06-15 00:06:32+02:00,sleep,392
3,sleep,complete,2011-06-15 03:38:29+02:00,sleep,13109
4,toilet,start,2011-06-15 03:38:44+02:00,toilet,13124
...,...,...,...,...,...
77,personalhygiene,complete,2011-06-15 23:19:40+02:00,personalhygiene,83980
78,sleep,start,2011-06-15 23:20:09+02:00,sleep,84009
79,sleep,complete,2011-06-16 07:00:40+02:00,sleep,25240
80,End,start,2011-06-16 07:00:40+02:00,End,25240


In [31]:
df["time:timestamp"]

0    2011-06-15 00:06:32+02:00
1    2011-06-15 00:06:32+02:00
2    2011-06-15 00:06:32+02:00
3    2011-06-15 03:38:29+02:00
4    2011-06-15 03:38:44+02:00
                ...           
77   2011-06-15 23:19:40+02:00
78   2011-06-15 23:20:09+02:00
79   2011-06-16 07:00:40+02:00
80   2011-06-16 07:00:40+02:00
81   2011-06-16 07:00:40+02:00
Name: time:timestamp, Length: 82, dtype: datetime64[ns, tzoffset(None, 7200)]

In [17]:
from sklearn.naive_bayes import GaussianNB

In [21]:
model = GaussianNB()
model.fit(df["seconds_since_midnight"].to_numpy().reshape(-1, 1), df["concept:name"])

GaussianNB()

In [22]:
model.predict(np.array([[1]]))

array(['sleep'], dtype='<U15')

In [23]:
filepath = "data/Real-life event logs - Hospital log_1_all/Hospital_log.xes\Hospital_log.xes"
xes_hospital = xes.xes_to_dataframes(filepath=filepath, load_traces=[824], keep_original=True)


In [33]:
xes_hospital[0]["events"]["time:timestamp"] = pd.to_datetime(xes_hospital[0]["events"]["time:timestamp"], utc=True)

xes_hospital[0]["events"]["time:timestamp"]

0      2006-09-18 23:00:00+00:00
1      2006-09-18 23:00:00+00:00
2      2006-09-18 23:00:00+00:00
3      2006-09-18 23:00:00+00:00
4      2006-09-18 23:00:00+00:00
                  ...           
1809   2007-06-27 23:00:00+00:00
1810   2007-08-07 23:00:00+00:00
1811   2007-11-25 23:00:00+00:00
1812   2007-11-25 23:00:00+00:00
1813   2008-02-12 23:00:00+00:00
Name: time:timestamp, Length: 1814, dtype: datetime64[ns, UTC]

In [34]:
add_seconds_since_midnight(xes_hospital[0]["events"])

,org:group,Number of executions,Specialism code,concept:name,Producer code,Section,Activity code,time:timestamp,lifecycle:transition,seconds_since_midnight
0,General Lab Clinical Chemistry,1,86,aanname laboratoriumonderzoek,CRLA,Section 4,370000,2006-09-18 23:00:00+00:00,complete,82800
1,General Lab Clinical Chemistry,1,86,aanname laboratoriumonderzoek,CRLA,Section 4,370000,2006-09-18 23:00:00+00:00,complete,82800
2,General Lab Clinical Chemistry,1,86,aanname laboratoriumonderzoek,CRLA,Section 4,370000,2006-09-18 23:00:00+00:00,complete,82800
3,General Lab Clinical Chemistry,1,86,bilirubine -geconjugeerd,CHE2,Section 4,370401,2006-09-18 23:00:00+00:00,complete,82800
4,General Lab Clinical Chemistry,1,86,bilirubine - totaal,CHE2,Section 4,370401C,2006-09-18 23:00:00+00:00,complete,82800
...,...,...,...,...,...,...,...,...,...,...
1809,Nursing ward,1,7,190205 klasse 3b a205,H5ZU,Section 2,613000,2007-06-27 23:00:00+00:00,complete,82800
1810,Obstetrics & Gynaecology clinic,1,7,vervolgconsult poliklinisch,SGNA,Section 2,411100,2007-08-07 23:00:00+00:00,complete,82800
1811,Obstetrics & Gynaecology clinic,1,7,vervolgconsult poliklinisch,SGNA,Section 2,411100,2007-11-25 23:00:00+00:00,complete,82800
1812,Obstetrics & Gynaecology clinic,1,7,administratief tarief - eerste pol,SGNA,Section 2,419100,2007-11-25 23:00:00+00:00,complete,82800


In [ ]:
def kde_generate1(df):
    kde = KernelDensity()
    kde.fit()